In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sentencefeaturesoov', 'crawl_emb_nocomp.pickle', 'jigsaw-unintended-bias-in-toxicity-classification', 'crawl_emb_processed_lz4.joblib', 'x-train-tokenized', 'crawl_emb_nocomp.joblib', 'crawl_emb_processed.joblib', 'bert-pretrained-models', 'fasttext-crawl-300d-2m', 'glove840b300dtxt', 'roov-crawl.pickle']


In [2]:
LSTM_UNITS = 128
batch_size = 512
n_seeds = 1
n_splits = 10
n_epochs = 20
EMBED_SIZE = 300
SUBGROUP_NEGATIVE_WEIGHT_COEF = 1
BACKGROUND_POSITIVE_WEIGHT_COEF = 0

MAX_LEN = 220

EMB_DROPOUT = 0.3
MIDDLE_DROPOUT = 0.3

DEBUG = False
if DEBUG:
    n_seeds = 2
    n_splits = 2
    n_epochs = 3
DEBUG_DATA_SIZE = 10000

In [3]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm import tqdm
from contextlib import contextmanager
from fastprogress import master_bar, progress_bar
from keras.preprocessing import sequence
from keras import preprocessing
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

tqdm.pandas()

Using TensorFlow backend.


In [4]:
%%time

import re
# これだと、'はembeddingに結構入ってるのに除外されちゃう。　よくないので ' だけ抜いた
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x: str) -> str:
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' {} '.format(punct))
    return x

# 9.9G


if DEBUG:
    DEBUG_DATA_SIZE = 10000
    train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', nrows=DEBUG_DATA_SIZE)
else:
    train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

x_train = train['comment_text'].apply(lambda x: clean_text(x))

CPU times: user 20.9 s, sys: 1.15 s, total: 22 s
Wall time: 22.4 s


In [27]:
x_train.apply(lambda x: len(x)).sum()

568691278

In [14]:
from textblob import TextBlob

In [19]:
list(x_train[:10])

["This is so cool .  It's like ,  'would you want your mother to read this ?  ? ' Really great idea ,  well done ! ",
 "Thank you !  !  This would make my life a lot less anxiety - inducing .  Keep it up ,  and don't let anyone get in your way ! ",
 'This is such an urgent design problem ;  kudos to you for taking it on .  Very impressive ! ',
 "Is this something I'll be able to install on my site ?  When will you be releasing it ? ",
 'haha you guys are a bunch of losers . ',
 'ur a sh * tty comment . ',
 'hahahahahahahahhha suck it . ',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem motivated by mostly by greed ;  no one should have the right to allow their animals destroy public land . ',
 "It was a great show .  Not a combo I'd of expected to be good together but it was . "]

In [8]:
str(TextBlob("This is so cool .").translate(to='ja'))

'これはとてもクールです 。'

In [16]:
en_ja_list = []
for text in x_train[:10]:
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='ja').translate(to='en')))
    except:
        en_ja_list.append(text)

In [10]:
str(TextBlob("This is so cool .").translate(to='ja').translate(to='en'))

'This is so cool.'

In [11]:
text

'FFFFUUUUUUUUUUUUUUU'

In [17]:
en_ja_list

['This is so cool. Like, "Do you want your mother to read this?" \'A really great idea, it worked!',
 "Thank you very much ! ! This makes my life far less disturbing-induced. Keep it and don't let someone get in your way!",
 'This is a very urgent design issue. I admire you for taking on it. Very impressive!',
 'Is this something I can install on my site? When will you release it?',
 'You all are a bunch of losers.',
 'Please add a comment.',
 'hahahahahahahahhha sucks it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The rancher seems to be motivated by greed by the Lord. You should not have the right to allow their animals to destroy the public land.',
 'It was a great show. Not a combo I was expecting a good together but it was good.']

In [20]:
en_ja_list = []
for text in x_train[:10]:
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='de').translate(to='en')))
    except:
        en_ja_list.append(text)

In [21]:
en_ja_list

["That is so cool . It's like: Do you want your mom to read this? ? 'Really great idea, well done!",
 'Many Thanks ! ! This would make my life much less fearful. Keep up the good work and do not let anyone get in your way!',
 'This is an urgent design problem. Kudos to you for accepting it. Very impressive !',
 'Can I install this on my website? When will you publish it?',
 'haha you are a few losers.',
 'Your comment *.',
 'hahahahahahahahhha suck it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem to be mainly motivated by greed. Nobody should have the right that his animals destroy public land.',
 'It was a great show. No combination that I expected to be good together, but it was.']

In [22]:
en_ja_list = []
for text in x_train[:10]:
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='fr').translate(to='en')))
    except:
        en_ja_list.append(text)

In [23]:
en_ja_list

["This is so cool . It's like, 'would you like your mother to read this? ? 'Really good idea, congratulations!",
 'Thank you ! ! That would make my life a lot less distressing - inducing. Continue like this, and do not let anyone bother you!',
 "It's such an urgent design problem. Congratulations to you for taking it. Very impressive !",
 'Can I install it on my site? When will you publish it?',
 'haha you are a group of losers.',
 'ur comment sh * tty.',
 'hahahahahahahahhha suck him.',
 'FFFFUUUUUUUUUUUUUUU',
 'Breeders seem motivated mainly by greed; no one should have the right to allow his animals to destroy public lands.',
 'It was a great show. I was not expecting a successful combo, but it was.']

In [24]:
en_ja_list = []
for text in x_train[:10]:
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='es').translate(to='en')))
    except:
        en_ja_list.append(text)

In [25]:
en_ja_list

["This is great . It's like, 'do you want your mother to read this? ? 'Very good idea, well done!",
 'Thank you ! ! This would make my life much less anxiety-inducing. Keep it up and do not let anyone get in your way!',
 'This is a design problem so urgent; Congratulations to you for assuming it. Very impressive !',
 'Is this something I can install on my site? When will you launch it?',
 'haha you are a group of losers.',
 'ur to sh * tty comentario.',
 'hahahahahahahahhha lo chupa',
 'FFFFUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU',
 'Farmers seem motivated mainly by greed; No one should have the right to allow their animals to destroy public lands.',
 'It was a great show. It was not a combo that I would have expected it to be good together, but it was.']

In [26]:
en_ja_list = []
for text in x_train[:10]:
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='zh').translate(to='en')))
    except:
        en_ja_list.append(text)
        
en_ja_list

['This is so cool. Like, \'Do you want your mother to read this? ? "Really great, good job!',
 "Thank you! ! This will make my life less anxious - induced. Stick to it, don't let anyone hinder you!",
 'This is such a pressing design issue; thank you for accepting it. Very impressive!',
 'Is this something I can install on my website? When did you release it?',
 'Haha, you are a group of losers.',
 'Your comment.',
 'Hahahahahahahahhha sucks it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem to be primarily driven by greed; no one should have the right to allow their animals to destroy public lands.',
 'This is a great show. Not a combination, I expect it to be good, but it is.']

In [30]:
with open('tmp.txt', 'w') as f:
    f.write('\n'.join(en_ja_list))